In [139]:
import pandas as pd
import pickle

In [94]:
raw = pd.read_csv(r"C:\Users\Dev Work\Documents\aicrowd\insurancepricing\python\training.csv")

In [107]:
data = raw[["vh_make_model", "claim_amount"]].copy()

data['claim'] = [1 if x > 0 else 0 for x in data['claim_amount']]

In [121]:
count = data.groupby("vh_make_model")['vh_make_model'].count()
claims = data.groupby("vh_make_model")['claim'].max()
avg_claim_cost = data.groupby("vh_make_model")['claim_amount'].mean()
sum_claim_cost = data.groupby("vh_make_model")['claim_amount'].sum()
claim_frequency = data.groupby("vh_make_model")['claim'].sum()
avg_frequency = (data.groupby("vh_make_model")['vh_make_model'].count() / len(data)) * 100

In [123]:
claims = [1 if claim > 0 else 0 for claim in claims]

risk_data = pd.DataFrame({"count":count, 
                        "frequency":avg_frequency,
                        "claim_ind": claims, 
                        "sum_claim_cost": sum_claim_cost,
                        "avg_claim_cost": avg_claim_cost,  
                        "claim_frequency": claim_frequency})

risk_data['burn_cost'] = risk_data['sum_claim_cost'] / risk_data['count']

In [126]:
risk_data['burn_cost'].describe()

count     975.000000
mean       92.246201
std       150.847998
min         0.000000
25%         0.000000
50%        42.379167
75%       131.344590
max      2143.412500
Name: burn_cost, dtype: float64

In [132]:
risk_data_filtered = risk_data[risk_data["claim_ind"] > 0 ]
risk_data_clean_models = risk_data[risk_data["claim_ind"] == 0 ]

In [140]:
risk_data_filtered['burn_cost']

with open('vh_burn_costs.pickle', 'wb') as target:
        pickle.dump(risk_data_filtered['burn_cost'], target)

In [135]:
risk_data_clean_models['burn_cost'].describe()

count    364.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: burn_cost, dtype: float64

In [ ]:
risk_rating = avg_claim_cost * (avg_frequency - avg_frequency.min())/(avg_frequency.max() - avg_frequency.min())
risk_rating.describe()
risk_rating.plot()